In [2]:
import pandas as pd
import numpy as np
import requests
import os 
from dotenv import load_dotenv
import requests
from tqdm import tqdm
import pickle
import json
pd.options.display.max_columns = None

In [3]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials ## para poner mis credenciales

# Importing pickle file

In [4]:
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [5]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [6]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.head()

,url,artist,track
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# spotify
Aquí está la docu: https://spotipy.readthedocs.io/en/2.22.0/#

Configuro el cliente y le metó las keys para acceder a la API de Spotify y poder sacar datos de las canciones que tengo

In [7]:
load_dotenv()

True

In [8]:
id = os.getenv("id")
secret = os.getenv("secret")

client_credentials_manager = SpotifyClientCredentials(client_id=id, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

See if Spotify is available

In [10]:
import tekore as tk

app_token = tk.request_client_token(id, secret)
spotify = tk.Spotify(app_token)

album = spotify.album('3RBULTZJ97bvVzZLpxcB0j')

TooManyRequests: Error in https://api.spotify.com/v1/albums/3RBULTZJ97bvVzZLpxcB0j:
429: 


# API calls

Primero voy a sacar el URI de mis canciones. Para ello uso el método de spotipy.search, que permite buscar con una query el nombre del artista y la canción.

In [10]:
## esto es para obtener una uri --> me voy a sacar los uris para mi data frame
artist = 'alan walker'
track = 'alone'
q = 'artist:{} track: {}'.format(artist, track)
results = sp.search(q=q, limit=1, type='track')
print(results['tracks']['items'][0]['uri'])

In [ ]:
results['tracks']['items'][0]['popularity']

In [ ]:
results['tracks']['items'][0]['album']['release_date']

In [ ]:
for index, row in tracks.iterrows():
    print(row['artist'],'---', row['track'])
    break

In [ ]:
tqdm.pandas()

uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'].split(',')[0], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    try:
        uri = res['tracks']['items'][0]['uri']
        #print(res['tracks']['items'][0]['uri'])
        uris.append(uri)
    except:
        uris.append('')
    #break

Getting URIs + popularity + release_date

In [ ]:
dict_popu_search = {
    'url':[],
    'uri':[],
    'popu':[],
    'date':[]
}

In [ ]:
tqdm.pandas()

#uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    q = 'artist:{} track: {}'.format(row['artist'].split(',')[0], row['track'])
    res = sp.search(q=q, limit=1, type='track')
    dict_popu_search['url'].append(row['url'])
    try:
        dict_popu_search['uri'].append(res['tracks']['items'][0]['uri'])
        dict_popu_search['popu'].append(res['tracks']['items'][0]['popularity'])
        dict_popu_search['date'].append(res['tracks']['items'][0]['album']['release_date'])
    except:
        dict_popu_search['uri'].append('')
        dict_popu_search['popu'].append('')
        dict_popu_search['date'].append('')
    #break

In [ ]:
pd.DataFrame(dict_popu_search)

In [ ]:
df_datos = pd.DataFrame(dict_popu_search)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

Voy a comprobar que la lista donde me he appendeado todo tiene la misma longitud que mi dataframe, para poder añadir los URIs como una columna

In [ ]:
uris[:3]

In [ ]:
print(len(uris), tracks.shape)

In [ ]:
tracks['URI'] = uris
tracks.head()

Ahora con el método `'audio_features'` de spotipy quiero sacar las propiedades que tienen las canciones. Para ello, tengo que aportar:
- `URI`

Y el resultado de cada una de mis consultas, lo almacenaré en una lista de listas que luego appendearé a mi dataframe.

In [ ]:
sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8')

In [ ]:
type(sp.audio_features('spotify:track:3OHfY25tqY28d16oZczHc8'))

In [ ]:
datos_uris = []
for index, row in tqdm(tracks.iterrows(), total = tracks.shape[0]):
    uri = sp.audio_features(row['URI'])
    datos_uris.append(uri)

Y tengo mis datos sobre las canciones, pero tengo que limpiar un poco para poder hacer el append.

In [ ]:
type(pd.DataFrame(datos_uris).iloc[0,0]) ## puedo sacar las claves para nombrar a mis columnas

In [ ]:
columnas = list(pd.DataFrame(datos_uris).iloc[0,0].keys())
columnas[:5]

In [ ]:
df_uris = pd.DataFrame(datos_uris)
df_uris[columnas] = df_uris[0].apply(pd.Series)
df_uris.head()

Lo junto ahora con mi dataframe inicial y limpio las columnas que no necesite.

In [ ]:
tracks = tracks.merge(df_uris, left_on = 'URI', right_on='uri', indicator=True)
tracks['_merge'].value_counts() ## compruebo que el cruce se haga con éxito

In [ ]:
tracks.drop([0, 'uri', '_merge'], axis = 1, inplace=True)
tracks.head()

llamando a la api

In [ ]:
SEARCH_PLAYLIST_ENDPOINT ='https://api.spotify.com/v1/search?type=playlist'
AUDIO_FEATURES_ENDPOINT = 'https://api.spotify.com/v1/audio-features/{id}'
TRACKS_ENDPOINT = 'https://api.spotify.com/v1/tracks/id'

In [ ]:
def search_playlist(name):
    path = 'token.json'
    with open(path) as t:
        token = json.load(t)
    myparams = {'type': 'playlilst'}
    myparams['q'] = name
    resp = requests.get(SEARCH_PLAYLIST_ENDPOINT, params=myparams, headers={"Authorization": "Bearer {}".format(token)})
    return resp.json()



In [ ]:
def get_audio_features(track_id):
    path = 'token.json'
    with open(path) as t:
        token = json.load(t)
    url = AUDIO_FEATURES_ENDPOINT.format(id=track_id)
    resp = requests.get(url, headers={"Authorization": "Bearer {}".format(secret)})
    return resp.json()


In [ ]:
get_audio_features(uris[0])

# Export

In [ ]:
with open('../data/pickle/spotify.pickle', 'wb') as data_spotify:
    pickle.dump(tracks, data_spotify)